In [1]:
!conda env list

# conda environments:
#
base                     /data/anaconda3
                         /home/student/miniconda3
                      *  /home/student/miniconda3/envs/hw1_env



In [2]:
!which python

/home/student/miniconda3/envs/hw1_env/bin/python


In [3]:
import sys
print(sys.executable)

/home/student/miniconda3/envs/hw1_env/bin/python


In [1]:
#import torch

In [5]:
## notebook madness
## original kernel uses correct python install most of the time, wrong kernel the other half
## sklearn was also not installed in correct env

In [6]:
!conda list

# packages in environment at /home/student/miniconda3/envs/hw1_env:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             4.5                       1_gnu  
anyio                     3.5.0                    pypi_0    pypi
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0           py38h7f8727e_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
attrs                     21.4.0             pyhd3eb1b0_0  
babel                     2.10.1                   pypi_0    pypi
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.11.1           py38h06a4308_0  
blas                      1.0                         mkl  
bleach                    5.0.0                    pypi_0    pypi
brotlipy                  0.7.0           py38h27cfd23_1003  
bzip2                     1.0.8                h7b6447c_0  
ca-

In [1]:
#import torch
#import torch.nn as nn

#from torch.optim import Adam

import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression 


In [2]:
data_list = os.listdir('data/train')

In [3]:
df_list = []
for i in data_list:
    df_temp = pd.read_csv("data/train/"+i,sep='|')
    df_list.append(df_temp)

In [4]:
len(df_list)

20000

In [5]:
def clip_df(df, label='SepsisLabel'):
    df = df.drop(labels=['Unit1', 'Unit2','EtCO2','Bilirubin_direct','TroponinI','Fibrinogen'], axis=1)
    index = df.loc[df[label] == 1].index
    if len(index) == 0:
        return df
    else:
        return df.loc[: index[0]]

In [6]:
new_df_list = []
for df in df_list:
    df_new = clip_df(df)
    #print(df_new)
    new_df_list.append(df_new)
    #break

In [7]:
values_dict = {}
#Age_list = []

for df in new_df_list:
    means = df.mean().dropna().to_dict()
    Age = means['Age']
    Gender = means['Gender']
    for column in means.keys():    
        val = means[column]
        if Gender not in values_dict.keys():
            values_dict[Gender] = {} 
        if column not in values_dict[Gender].keys():
            values_dict[Gender][column] = []
        values_dict[Gender][column].append((Age,val))

In [8]:
coef = {}
for g in values_dict:
    coef[g] = {}
    for key,vals in values_dict[g].items():
        X = np.array([x[0] for x in vals])
        y = np.array([x[1] for x in vals])
        reg = LinearRegression().fit(X.reshape(-1,1), y)
        coef[g][key] = [reg.coef_[0],reg.intercept_]

In [9]:
y = []
X = []
#Z = 0
for df in new_df_list:
    label = df.iloc[-1]['SepsisLabel']
    means = df.mean().to_dict()
    
    df_nu = (1-df.isna().mean())
    df_nu = df_nu.drop(labels=['Age', 'Gender','HospAdmTime','ICULOS','SepsisLabel'])
    
    for key in means.keys():
        if np.isnan(means[key]):
            means[key] = coef[means['Gender']][key][0]*means['Age'] + coef[means['Gender']][key][1]
        
    y.append(label)
    means.pop('SepsisLabel',None)
    means_list = list(means.values())
    means_list.extend(list(df_nu))
    X.append(means_list)

In [10]:
x = np.array(X)
y = np.array(y)

data_num = x.shape[0]
feat_num = x.shape[1]

In [11]:
clf = LogisticRegression(max_iter=2000,verbose=1,solver='liblinear').fit(x, y)

[LibLinear]iter  1 act 7.956e+03 pre 7.178e+03 delta 4.915e-03 f 1.386e+04 |g| 3.488e+06 CG   2
iter  2 act 6.457e+02 pre 5.650e+02 delta 4.915e-03 f 5.907e+03 |g| 6.882e+05 CG   2
cg reaches trust region boundary
iter  3 act 1.249e+02 pre 1.540e+02 delta 4.915e-03 f 5.262e+03 |g| 1.484e+05 CG   4
cg reaches trust region boundary
iter  4 act 8.535e+01 pre 8.702e+01 delta 7.129e-03 f 5.137e+03 |g| 7.313e+04 CG   4
cg reaches trust region boundary
iter  5 act 6.000e+01 pre 6.044e+01 delta 1.241e-02 f 5.051e+03 |g| 1.722e+04 CG   4
cg reaches trust region boundary
iter  6 act 8.309e+01 pre 8.574e+01 delta 1.944e-02 f 4.991e+03 |g| 2.377e+04 CG   5
cg reaches trust region boundary
iter  7 act 5.059e+01 pre 5.101e+01 delta 1.944e-02 f 4.908e+03 |g| 1.847e+04 CG   6
cg reaches trust region boundary
iter  8 act 1.785e+01 pre 1.818e+01 delta 2.113e-02 f 4.858e+03 |g| 1.093e+04 CG   8
cg reaches trust region boundary
iter  9 act 1.233e+01 pre 1.181e+01 delta 2.268e-02 f 4.840e+03 |g| 3.963e+03 

In [12]:
data_list_test = os.listdir('data/test')

df_list_test = []
for i in data_list_test:
    df_temp = pd.read_csv("data/test/"+i,sep='|')
    df_list_test.append(df_temp)
    

new_df_list_test = []
for df in df_list_test:
    df_new = clip_df(df)
    new_df_list_test.append(df_new)
    
    
y_test = []
X_test = []
#Z = 0
for df in new_df_list_test:
    label = df.iloc[-1]['SepsisLabel']
    means = df.mean().to_dict()
    
    df_nu = (1-df.isna().mean())
    df_nu = df_nu.drop(labels=['Age', 'Gender','HospAdmTime','ICULOS','SepsisLabel'])
    
    for key in means.keys():
        if np.isnan(means[key]):
            means[key] = coef[means['Gender']][key][0]*means['Age'] + coef[means['Gender']][key][1]
        
    y_test.append(label)
    means.pop('SepsisLabel',None)
    means_list = list(means.values())
    means_list.extend(list(df_nu))
    X_test.append(means_list)

In [13]:
x_test = np.array(X_test)
y_test = np.array(y_test)

data_num = x_test.shape[0]
feat_num = x_test.shape[1]

In [14]:
print(x_test.shape)

(10000, 64)


In [17]:
y_pred_test = clf.predict(x_test)

In [19]:
f1_score(y_test, y_pred_test, average='macro')

0.5797071250282634

In [21]:
f1_score(y_test, y_pred_test, average='binary')

0.19661016949152543

In [22]:
print(x.shape, x_test.shape)

(20000, 64) (10000, 64)


In [23]:
clf.coef_

array([[ 1.46554652e-02,  4.18049159e-02,  3.43574764e-01,
         6.35682338e-03, -2.19765265e-02,  8.64702315e-04,
         5.32327896e-02,  9.50686582e-02, -4.22636281e-02,
        -7.87157112e-04, -1.91235372e+00, -1.96199561e-02,
         3.31649793e-03, -9.31846526e-05,  4.88459390e-03,
        -6.85640313e-04, -1.61449680e-01, -5.24828322e-03,
         7.32851366e-02,  1.26068794e-03, -1.50289072e-01,
         2.69965939e-01, -3.68409806e-02, -2.55801042e-01,
         9.99787069e-03,  3.84495516e-02, -1.24232637e-01,
        -1.07973370e-04,  1.61645736e-02,  6.77460422e-05,
         5.72612258e-03,  1.30016587e-01, -6.31292954e-04,
         3.70860486e-02, -4.30111887e+00,  2.91110690e-01,
        -1.07538206e+00, -2.16860202e+00,  2.66177868e+00,
         1.78595582e-01, -9.33241535e-01,  3.06113957e-01,
        -2.14384869e-01,  3.80985901e+00, -1.22197699e-01,
         8.83780516e-01, -7.58745803e-01,  1.64641893e+00,
        -1.80414743e-01,  1.21828209e+00,  1.14391289e+0